목표 : 부실 Labeling
================
조건 1 : 이자보상배율 3년 연속 1 미만   
조건 2 : 영업현금흐름 3년 연속 음수   
조건 3 : 감사의견 거절 혹은 부적절   

import modules

In [2]:
import pandas as pd

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\Users\LG\2ndProject\비상장\2.데이터전처리


환경설정

In [3]:
pd.options.display.float_format = "{:.3f}".format

read csv files

In [4]:
audit = pd.read_csv("../datasets/비상장_이자보상배율.csv", index_col=0)
audit = audit[
    [
        "회사명",
        "거래소코드",
        "회계년도",
        "이자보상배율(이자비용)",
        "[U01250010]   이자비용(천원)",
        "[U01230000]영업이익(손실)(천원)",
    ]
]
audit

,회사명,거래소코드,회계년도,이자보상배율(이자비용),[U01250010] 이자비용(천원),[U01230000]영업이익(손실)(천원)
0,(유)유맥에프앤알,14534,2012/12,4.840,222557.000,1078114.000
1,(유)유맥에프앤알,14534,2013/12,8.280,176881.000,1464673.000
2,(유)유맥에프앤알,14534,2014/12,9.580,159353.000,1526416.000
3,(유)유맥에프앤알,14534,2015/12,6.680,133414.000,890788.000
4,(유)유맥에프앤알,14534,2016/12,10.630,183594.000,1951647.000
...,...,...,...,...,...,...
243679,파라뷰플러스주식회사,73160,2018/12,-5.830,685240.000,-3994193.000
243680,파라뷰플러스주식회사,73160,2019/12,-1.300,881290.000,-1144716.000
243681,파라뷰플러스주식회사,73160,2020/12,-7.360,382417.000,-2815894.000
243682,파라뷰플러스주식회사,73160,2021/12,-0.820,1325927.000,-1083558.000


# Labeling 전 처리

In [5]:
# 회계월이 12월인 데이터만 남기고 회계년을 넣는다.
audit["회계년도"] = pd.to_datetime(audit["회계년도"]) # 데이터 형식을 날짜로 변환
audit = audit[audit["회계년도"].dt.month == 12].reset_index(drop=True) # 12월인 데이터만 남긴다
audit["회계년도"] = audit["회계년도"].dt.year # 회계년만 넣는다

In [6]:
# 영업손익이 0이거나 음수이면 이자보상배율에 0을 넣는다.
audit.loc[
    (audit["이자보상배율(이자비용)"] == 999999999) & (audit["[U01230000]영업이익(손실)(천원)"] < 0),
    "이자보상배율(이자비용)",
] = 0

In [7]:
# 부실판단에 필요한 데이터를 불러온다
df2 = pd.read_csv("../datasets/비상장_부실판단.csv", encoding='cp949', index_col=0)
df2.head(3)

,회사명,거래소코드,회계년도,감사의견코드,[U01410000]영업활동으로 인한 현금흐름(*)(천원),이자보상배율(이자비용)
0,(유)가든주류,97082,Dec-19,NaN,138498.000,3.170
1,(유)가든주류,97082,Dec-20,QS,-341826.000,1.290
2,(유)가든주류,97082,Dec-21,UQ,130657.000,0.290


In [8]:
# 회계월이 12월인 데이터만 남기고 회계년을 넣는다. 
df2["회계년도"] = pd.to_datetime(df2["회계년도"], format="%b-%y")
df2 = df2[df2["회계년도"].dt.month == 12].reset_index(drop=True)
df2["회계년도"] = df2["회계년도"].dt.year

In [9]:
# 데이터 합치기
unlisted_merged_df = pd.merge(
    df2, audit[["거래소코드", "회계년도", "이자보상배율(이자비용)"]], on=["거래소코드", "회계년도"], how="left"
)

In [10]:
unlisted_merged_df.columns

Index(['회사명', '거래소코드', '회계년도', '감사의견코드', '[U01410000]영업활동으로 인한 현금흐름(*)(천원)',
       '이자보상배율(이자비용)_x', '이자보상배율(이자비용)_y'],
      dtype='object')

In [11]:
# 중복된 컬럼을 제거
unlisted_df = unlisted_merged_df[
    [
        "회사명",
        "거래소코드",
        "회계년도",
        "감사의견코드",
        "[U01410000]영업활동으로 인한 현금흐름(*)(천원)",
        "이자보상배율(이자비용)_y",
    ]
]

In [12]:
# 컬럼 이름 변경
unlisted_df.rename(columns={"이자보상배율(이자비용)_y": "이자보상배율(이자비용)"}, inplace=True)

# 부실 판단

In [13]:
concat_df = pd.DataFrame()

##### 감사의견코드 부실 정의

In [14]:
df = unlisted_df.copy()

In [15]:
# 감사부실 컬럼 생성
df["감사부실"] = None

In [16]:
code_df = pd.DataFrame()

In [24]:
# 감사 Labeling
for i in range(len(df)):
    if df["감사의견코드"][i] in ["AG", "DI", "DS", "DU"]:
        df["감사부실"][i] = 1
    elif pd.isna(df.loc[i, "감사의견코드"]):
        df["감사부실"][i] = None
    else:
        df["감사부실"][i] = 0

In [25]:
# 사용할 감사부실 회사 최초부실
bank1 = df.loc[df["감사부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
bank1_com = bank1[["거래소코드", "회계년도"]]
bank1_com

,거래소코드,회계년도
3,97082,2022
15,40736,2012
42,61907,2016
98,100876,2021
141,43596,2015
...,...,...
251795,20555,2018
251806,20556,2018
251822,41249,2013
251906,61473,2012


In [27]:
bank1_com["회계년도"]

3         2022
15        2012
42        2016
98        2021
141       2015
          ... 
251795    2018
251806    2018
251822    2013
251906    2012
251945    2014
Name: 회계년도, Length: 3843, dtype: int32

In [ ]:
year_index = test[test["회계년도"] == year].index

In [26]:
# 3년 연속 없는 데이터 제거
nan_list = []

for code, year in zip(bank1_com["거래소코드"], bank1_com["회계년도"]): # code는 거래소코드를 넣는다. year는 회계년을 넣는다.
    test = df[df["거래소코드"] == code] # test는 감사부실인 기업의 거래소코드
    year_index = test[test["회계년도"] == year].index # year_index는 감사부실인 회계년도의 인덱스
    if (code != df["거래소코드"].iloc[year_index[0] - 2]) | (year - 2) != df["회계년도"].iloc[
        year_index[0] - 2
    ]: 
        nan_list.append(code)

print(nan_list)

[40736, 100876, 53950, 79480, 68378, 106039, 60884, 65136, 57027, 45504, 54259, 76212, 62262, 82159, 86939, 29883, 98468, 83246, 72438, 46819, 57184, 57009, 68027, 87582, 53278, 70260, 65141, 89806, 17508, 61780, 71029, 66966, 47173, 87004, 85680, 58058, 88501, 60932, 47098, 47102, 22140, 95853, 52178, 46163, 26267, 91761, 95736, 57270, 93738, 42284, 61234, 48220, 50187, 103717, 64487, 12629, 53497, 66879, 89176, 48216, 48750, 89387, 71061, 46061, 68377, 41730, 71142, 50048, 59094, 58290, 58288, 62476, 54029, 53503, 88671, 29762, 13564, 10211, 106520, 29922, 97004, 50653, 45724, 53733, 53236, 38812, 66824, 59411, 49677, 87303, 77367, 104678, 46790, 67332, 44685, 29533, 65368, 52154, 70141, 59239, 53001, 99107, 12947, 63220, 76285, 40643, 59906, 29500, 85910, 49584, 93881, 29393, 106423, 58378, 56359, 63284, 75995, 88140, 71014, 13274, 57600, 29727, 61331, 90010, 77527, 29516, 10569, 104886, 83074, 41871, 45721, 66157, 44436, 51942, 40994, 81706, 91115, 97012, 96255, 70839, 76944, 65061

In [28]:
# df[df["거래소코드"] == year_nan_list]
for code in nan_list:
    df = df[df["거래소코드"] != code]

In [29]:
grouped = df.groupby("거래소코드")

In [30]:
year_nan_list = []

for code, group in grouped:
    len_group = len(group)
    if len_group < 3:
        year_nan_list.append(code)

print(year_nan_list)

[10177, 10401, 10590, 10631, 10658, 10689, 10797, 10832, 10861, 10945, 10961, 11022, 11327, 11452, 11459, 11500, 11704, 11711, 11733, 11866, 11895, 11982, 12172, 12206, 12289, 12369, 12371, 12501, 12579, 12704, 12849, 12920, 12961, 13042, 13064, 13085, 13099, 13146, 13236, 13327, 13386, 13409, 13427, 13460, 13495, 13563, 13611, 13685, 13747, 13803, 13840, 13861, 13952, 14037, 14073, 14199, 14278, 14430, 14439, 14469, 14513, 14568, 14649, 14663, 14711, 14715, 14742, 14750, 14800, 14805, 14821, 14871, 14882, 14990, 15004, 15036, 15185, 15259, 15301, 15367, 15476, 15530, 15577, 15601, 15622, 15648, 15800, 15844, 16102, 16282, 16344, 16347, 16467, 16501, 16555, 16627, 16692, 16696, 16700, 16880, 16882, 17126, 17151, 17189, 17372, 17385, 17481, 17605, 17619, 17643, 17739, 17800, 17831, 17834, 17970, 17983, 18047, 18056, 18115, 18122, 18129, 18130, 18210, 18316, 18400, 18454, 18550, 18645, 18736, 18791, 18829, 18840, 18867, 18900, 18927, 18944, 19055, 19061, 19065, 19111, 19150, 19261, 19266

In [31]:
# df[df["거래소코드"] == year_nan_list]
for code in year_nan_list:
    df = df[df["거래소코드"] != code]

In [32]:
df = df.reset_index(drop=True)

In [33]:
# # 사용할 감사부실 회사
insolvent_com = df.loc[df["감사부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
insolvent_codes = insolvent_com["거래소코드"].unique()

non_insolvent_com = df[~df["거래소코드"].isin(insolvent_codes)]  # 부실선정 기업 제거
non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

non_insolvent_com.isna().sum()

print(len(insolvent_codes))
print(len(non_insolvent_com) - len(non_insolvent_com.isna().sum()))
print(len(non_insolvent_com.isna().sum()))
print(len(df["거래소코드"].unique()))

1769
30551
7
32327


In [34]:
code_df = pd.concat([insolvent_com, non_insolvent_com])

##### 이자보상배율 부실정의

In [35]:
df = unlisted_df.copy()

In [36]:
ratio_df = pd.DataFrame()

In [37]:
df["이보배부실"] = None

In [38]:
grouped = df.groupby("거래소코드")

In [82]:
for code, group in grouped:
    # print(code, group)
    group_len = len(group)
    # print(group_len)
    for i in range(group_len - 2):
        cond1 = group.iloc[i]["이자보상배율(이자비용)"] < 1
        cond2 = group.iloc[i + 1]["이자보상배율(이자비용)"] < 1
        cond3 = group.iloc[i + 2]["이자보상배율(이자비용)"] < 1
        # print(
        #     group.iloc[i]["이자보상배율(이자비용)"],
        #     group.iloc[i + 1]["이자보상배율(이자비용)"],
        #     group.iloc[i + 2]["이자보상배율(이자비용)"],
        # )
        # print("==========")
        cond4 = pd.isnull(group.iloc[i]["이자보상배율(이자비용)"])
        cond5 = pd.isnull(group.iloc[i + 1]["이자보상배율(이자비용)"])
        cond6 = pd.isnull(group.iloc[i + 2]["이자보상배율(이자비용)"])

        if cond1 & cond2 & cond3:
            # print(group["거래소코드"])
            df.loc[group.index[i + 2], "이보배부실"] = 1
        elif cond4 or cond5 or cond6:
            df.loc[group.index[i + 2], "이보배부실"] = None
        else:
            df.loc[group.index[i + 2], "이보배부실"] = 0

        # print(group[i]['이자보상배율(이자비용)'])

In [83]:
## 사용할 이보배부실 회사
bank2 = df.loc[df["이보배부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
bank2_com = bank2[["거래소코드", "회계년도"]]
bank2_com

,거래소코드,회계년도
18,93640,2021
24,81962,2018
33,64118,2016
43,61907,2017
85,58996,2014
...,...,...
251908,61473,2014
251945,60017,2014
251949,98297,2022
251956,49187,2014


In [84]:
nan_list = []

for code, year in zip(bank2_com["거래소코드"], bank2_com["회계년도"]):
    test = df[df["거래소코드"] == code]
    year_index = test[test["회계년도"] == year].index
    if (code != df["거래소코드"].iloc[year_index[0] - 2]) | (year - 2) != df["회계년도"].iloc[
        year_index[0] - 2
    ]:
        nan_list.append(code)

print(nan_list)

[38630, 65136, 63203, 91141, 90871, 49566, 57167, 41866, 61234, 58206, 71058, 61388, 69310, 57064, 28818, 47415, 29415, 60547, 51650, 64352, 69474, 76233, 29922, 61318, 67679, 81977, 95753, 64643, 53001, 10567, 87963, 28529, 51181, 39497, 13188, 69190, 68174, 67078, 81720, 70194, 56869, 80938, 59516, 96739, 67254, 46993, 74239, 23341, 56847, 70118, 14031, 90273, 70347, 51870, 42091, 59728, 89742, 10361, 74718, 61605, 46414, 50828, 69793, 73869, 77911, 39558, 49092, 45601, 63841, 90973, 90630, 38874, 64764, 87729, 76818, 48927, 13702, 30356, 38915, 13915, 94663, 13922, 89589, 81101, 62701, 83418, 95472, 64517, 84702, 16309, 78699, 64258, 92701, 49867, 81343, 41448, 86384, 45820, 82973, 90417, 80453, 73910, 61166, 83150, 90366, 59462, 61024, 90591, 51773, 77351, 87154, 47525, 88030, 15125, 15148, 28835, 18038, 69605, 18046, 40954, 43028, 76228, 68954, 57119, 46457, 56951, 17274, 70785, 62043, 73948, 51385, 72709, 52269, 71000, 41958, 15507, 80792, 15617, 59184, 96742, 69974, 77995, 52452

In [85]:
# df[df["거래소코드"] == year_nan_list]
for code in nan_list:
    df = df[df["거래소코드"] != code]

In [86]:
year_nan_list = []

for code, group in grouped:
    # print(code, group)
    # print(len(group))
    len_group = len(group)
    if len_group < 3:
        year_nan_list.append(code)
print(year_nan_list)

[10177, 10322, 10401, 10590, 10631, 10652, 10658, 10689, 10797, 10832, 10861, 10945, 10961, 11022, 11231, 11327, 11400, 11452, 11459, 11500, 11704, 11711, 11733, 11866, 11895, 11982, 12172, 12206, 12289, 12369, 12371, 12501, 12579, 12650, 12704, 12849, 12920, 12947, 12961, 13042, 13064, 13085, 13099, 13146, 13236, 13274, 13327, 13386, 13409, 13427, 13460, 13469, 13495, 13563, 13564, 13589, 13611, 13685, 13689, 13747, 13803, 13840, 13861, 13870, 13952, 13988, 14037, 14073, 14117, 14199, 14278, 14430, 14439, 14456, 14469, 14513, 14568, 14649, 14663, 14711, 14712, 14715, 14742, 14750, 14800, 14805, 14821, 14871, 14882, 14990, 15004, 15036, 15185, 15259, 15301, 15367, 15476, 15496, 15530, 15577, 15601, 15622, 15648, 15738, 15800, 15844, 15911, 16102, 16206, 16282, 16344, 16347, 16467, 16501, 16555, 16627, 16692, 16696, 16700, 16765, 16793, 16880, 16882, 16952, 17117, 17119, 17126, 17151, 17189, 17204, 17372, 17385, 17421, 17422, 17478, 17481, 17507, 17508, 17578, 17605, 17619, 17643, 17739

In [87]:
# df[df["거래소코드"] == year_nan_list]
for code in year_nan_list:
    df = df[df["거래소코드"] != code]

In [88]:
# # 사용할 감사부실 회사
insolvent_com = df.loc[df["이보배부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
insolvent_codes = insolvent_com["거래소코드"].unique()

non_insolvent_com = df[~df["거래소코드"].isin(insolvent_codes)]  # 부실선정 기업 제거
non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

non_insolvent_com.isna().sum()

print(len(insolvent_codes))
print(len(non_insolvent_com) - len(non_insolvent_com.isna().sum()))
print(len(non_insolvent_com.isna().sum()))
print(len(df["거래소코드"].unique()))

11328
21216
7
32551


/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_80136/1599080698.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)


In [89]:
ratio_df = pd.concat([insolvent_com, non_insolvent_com])

##### 현금흐름 부실

flow_df = pd.DataFrame()

In [90]:
df = unlisted_df.copy()

In [91]:
df["현금흐름부실"] = None

In [92]:
grouped = df.groupby("거래소코드")

In [93]:
for code, group in grouped:
    # print(code, group)
    group_len = len(group)
    # print(group_len)
    for i in range(group_len - 2):
        cond1 = group.iloc[i]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"] < 0
        cond2 = group.iloc[i + 1]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"] < 0
        cond3 = group.iloc[i + 2]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"] < 0
        # print(
        #     group.iloc[i]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"],
        #     group.iloc[i + 1]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"],
        #     group.iloc[i + 2]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"],
        # )
        # print("==========")
        cond4 = pd.isnull(group.iloc[i]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"])
        cond5 = pd.isnull(group.iloc[i + 1]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"])
        cond6 = pd.isnull(group.iloc[i + 2]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"])

        if cond1 & cond2 & cond3:
            # print(group["거래소코드"])
            df.loc[group.index[i + 2], "현금흐름부실"] = 1
        elif cond4 or cond5 or cond6:
            df.loc[group.index[i + 2], "현금흐름부실"] = None
        else:
            df.loc[group.index[i + 2], "현금흐름부실"] = 0

        # print(group[i]['[U01410000]영업활동으로 인한 현금흐름(*)(천원)'])

In [94]:
# # 사용할 현금흐름부실 회사
bank3 = df.loc[df["현금흐름부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
bank3_com = bank3[["거래소코드", "회계년도"]]
bank3_com

,거래소코드,회계년도
26,81962,2020
66,63900,2017
74,101458,2022
85,58996,2014
94,30661,2015
...,...,...
251729,20550,2019
251804,20556,2016
251905,101000,2022
251949,98297,2022


In [95]:
nan_list = []

for code, year in zip(bank3_com["거래소코드"], bank3_com["회계년도"]):
    test = df[df["거래소코드"] == code]
    year_index = test[test["회계년도"] == year].index
    if (code != df["거래소코드"].iloc[year_index[0] - 2]) | (year - 2) != df["회계년도"].iloc[
        year_index[0] - 2
    ]:
        nan_list.append(code)

print(nan_list)

[65136, 40058, 63203, 90871, 57167, 61289, 65646, 71058, 69310, 51650, 69474, 61318, 87409, 87963, 44327, 39497, 13188, 75995, 69618, 51942, 65775, 80991, 80938, 72668, 74239, 14031, 86254, 70842, 80448, 51870, 71252, 42091, 89742, 58917, 77069, 46414, 50828, 63841, 89703, 51283, 48782, 90630, 38915, 13915, 75060, 69822, 66683, 67058, 66961, 90417, 83150, 81291, 30081, 49157, 90366, 61024, 69365, 66864, 87154, 15125, 15148, 97076, 18038, 69605, 40954, 76228, 29437, 15417, 63860, 69974, 64097, 77995, 92030, 65144, 15810, 15833, 77954, 54664, 78273, 54788, 59762, 85056, 96850, 47033, 77330, 40849, 64592, 43315, 89125, 83407, 94983, 74045, 72583, 82385, 89664, 64279, 67061, 76614, 40126, 67915, 72645, 58973, 41023, 73578, 58773, 46058, 62041, 67840, 66982, 60747, 82876, 52228, 86094, 88681, 93505, 21089, 67926, 87420, 69673, 56388, 30656, 91924, 64797, 25550, 46034, 59279, 53000, 80386, 72867, 96889, 88013, 59208, 17011, 73805, 87872, 81905, 91472, 44770, 85404, 95322, 50546, 70424, 85937

In [96]:
# df[df["거래소코드"] == year_nan_list]
for code in nan_list:
    df = df[df["거래소코드"] != code]

In [97]:
year_nan_list = []

for code, group in grouped:
    # print(code, group)
    # print(len(group))
    len_group = len(group)
    if len_group < 3:
        year_nan_list.append(code)
print(year_nan_list)

[10177, 10322, 10401, 10590, 10631, 10652, 10658, 10689, 10797, 10832, 10861, 10945, 10961, 11022, 11231, 11327, 11400, 11452, 11459, 11500, 11704, 11711, 11733, 11866, 11895, 11982, 12172, 12206, 12289, 12369, 12371, 12501, 12579, 12650, 12704, 12849, 12920, 12947, 12961, 13042, 13064, 13085, 13099, 13146, 13236, 13274, 13327, 13386, 13409, 13427, 13460, 13469, 13495, 13563, 13564, 13589, 13611, 13685, 13689, 13747, 13803, 13840, 13861, 13870, 13952, 13988, 14037, 14073, 14117, 14199, 14278, 14430, 14439, 14456, 14469, 14513, 14568, 14649, 14663, 14711, 14712, 14715, 14742, 14750, 14800, 14805, 14821, 14871, 14882, 14990, 15004, 15036, 15185, 15259, 15301, 15367, 15476, 15496, 15530, 15577, 15601, 15622, 15648, 15738, 15800, 15844, 15911, 16102, 16206, 16282, 16344, 16347, 16467, 16501, 16555, 16627, 16692, 16696, 16700, 16765, 16793, 16880, 16882, 16952, 17117, 17119, 17126, 17151, 17189, 17204, 17372, 17385, 17421, 17422, 17478, 17481, 17507, 17508, 17578, 17605, 17619, 17643, 17739

In [98]:
# df[df["거래소코드"] == year_nan_list]
for code in year_nan_list:
    df = df[df["거래소코드"] != code]

In [99]:
# # 사용할 감사부실 회사
insolvent_com = df.loc[df["현금흐름부실"] == 1].drop_duplicates(
    subset=["거래소코드"], keep="first"
)
insolvent_codes = insolvent_com["거래소코드"].unique()

non_insolvent_com = df[~df["거래소코드"].isin(insolvent_codes)]  # 부실선정 기업 제거
non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

non_insolvent_com.isna().sum()

print(len(insolvent_codes))
print(len(non_insolvent_com) - len(non_insolvent_com.isna().sum()))
print(len(non_insolvent_com.isna().sum()))
print(len(df["거래소코드"].unique()))

6960
25840
7
32807


/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_80136/2403248965.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)


In [100]:
flow_df = pd.concat([insolvent_com, non_insolvent_com])

##### 데이터프레임 concat ( code_df, ratio_df, flow_df )

In [101]:
df_list = [code_df, ratio_df, flow_df]
audit = pd.concat(df_list)

In [102]:
audit = audit.sort_values(by=["거래소코드", "회계년도"], ascending=True)

In [103]:
# 상장 부실판단
audit["부실판단"] = None
cond1 = (audit["감사부실"] == 1) | (audit["이보배부실"] == 1) | (audit["현금흐름부실"] == 1)
cond2 = (
    (~cond1) & (audit["감사부실"] == 0) | (audit["이보배부실"] == 0) | (audit["현금흐름부실"] == 0)
)

audit.loc[cond1, "부실판단"] = 1
audit.loc[cond2, "부실판단"] = 0

In [104]:
## 상장기업
# 부실기업 데이터만 출력
bank1 = audit[audit["부실판단"] == 1]
bank1.drop_duplicates(subset="거래소코드", keep="first", inplace=True)

# 정상기업 데이터만 남기기
bank1_code = bank1["거래소코드"].unique()  # 부실로 선정된 기업의 거래소코드
non_bank1 = audit[~audit["거래소코드"].isin(bank1_code)]  # 부실선정 기업 제거
non_bank1.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

# 부실기업과 정상기업 데이터프레임 합치기
final1 = pd.concat([bank1, non_bank1])
final1.reset_index(drop=True, inplace=True)

/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_80136/4122435525.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank1.drop_duplicates(subset="거래소코드", keep="first", inplace=True)
/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_80136/4122435525.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_bank1.drop_duplicates(subset="거래소코드", keep="last", inplace=True)


In [105]:
final1 = final1[["회사명", "거래소코드", "회계년도", "부실판단"]]

In [106]:
final1.to_csv("../datasets/Labeling/비상장_부실_Labeling.csv")